# Introduction

This notebook demonstrates how to accelerate topic modeling at scale using <b>BERTopic</b> with <b>NVIDIA GPU acceleration</b>. We explore how drop-in GPU support for <b>UMAP</b> and <b>HDBSCAN</b> via <b>cuML</b> can significantly reduce runtime without changing existing code.

Our example uses the [<b>Amazon All Beauty Reviews dataset</b>](https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/review_categories/All_Beauty.jsonl.gz) , which contains over <b>700K product reviews </b>. To reduce redundancy and improve clustering quality, we remove duplicate embeddings generated from preprocessed text using a transformer model. This brings the dataset down to approximately <b>638K unique entries</b>.

We then walk through a BERTopic pipeline using GPU-accelerated components to extract interpretable topics and visualize them. The dramatic speedup from CPU to GPU execution highlights the impact of GPU-powered NLP workflows. Note that this timing reflects a single run on a specific system configuration and may vary across different hardware setups.


# BERTopic Introduction

A diagram showing BERTopic's workflow:
<p style="text-align: center;">
  <img src="BERTopic diagram.png" width="750">
</p>


# Load and Preprocess Text

In [1]:
%load_ext cudf.pandas
import pandas as pd

Uncomment the code to load the dataset

In [2]:
# !wget https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/review_categories/All_Beauty.jsonl.gz --no-check-certificate

In [3]:
path = "All_Beauty.jsonl.gz"
data = pd.read_json(path, lines=True)
amazon_review = data.text.tolist()

In [4]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
    
# preprocess each text string 
preprocessed_texts = [preprocess_text(str(review)) for review in amazon_review]

In [5]:
print(len(amazon_review))
print(len(preprocessed_texts))

701528
701528


# Remove Duplicates

In [6]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(preprocessed_texts, show_progress_bar=False)

In [7]:
pd.DataFrame(pd.DataFrame(embeddings).value_counts()).head(10)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,count
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,
-0.033284,0.036665,0.036700,0.002700,0.001214,-0.044336,0.093854,0.038204,0.017783,0.001686,-0.008809,0.095864,0.003953,0.046457,-0.096746,-0.019579,-0.027649,-0.040081,-0.010177,-0.006142,-0.126816,0.013632,0.018427,0.009959,0.021555,-0.015510,0.023604,0.022434,-0.106529,-0.151864,-0.081125,0.024483,0.062927,0.008254,-0.104181,0.018431,0.016773,0.021700,-0.000510,0.021278,0.000210,-0.038646,0.002999,0.007322,0.003811,0.139792,-0.051353,0.006956,0.091817,0.005849,-0.011273,-0.085304,-0.015567,-0.051153,-0.026636,-0.012098,-0.046495,0.058402,0.010580,-0.076558,0.083992,0.056459,-0.046907,-0.014359,0.041535,-0.031042,-0.020871,-0.022688,-0.027590,-0.000406,-0.051000,0.050033,0.038518,-0.059188,-0.017704,0.023755,0.041485,-0.054027,0.007638,-0.045328,-0.009470,0.027472,-0.083163,0.090288,-0.074420,-0.094588,0.060015,0.035189,0.006835,0.011995,-0.002648,0.014584,0.012122,-0.023777,-0.097836,-0.023492,0.022663,-0.103430,-0.082450,0.159732,-0.037537,-0.049349,-0.006605,0.013648,-0.042524,-0.042881,-0.009396,0.067207,-0.050732,-0.088599,-0.032326,-0.005139,0.047381,0.009592,0.097082,-0.030100,0.037802,0.058862,0.038647,-0.007534,0.031677,0.052224,0.000689,-0.005144,-0.046224,-0.097170,0.028554,-3.363939e-33,0.037874,-0.062231,0.021175,-0.002563,0.128560,0.006064,0.054217,-0.025635,-0.089121,0.094719,0.050698,-0.009840,0.003257,0.059281,-0.006811,-0.100923,-0.079947,0.022360,0.063747,0.018769,0.056665,-0.006072,-0.031042,-0.030039,-0.023243,-0.008673,0.042071,-0.084638,0.097085,0.007209,-0.016889,0.037316,0.020978,-0.002157,-0.032216,-0.084836,-0.016991,-0.015201,0.007124,0.047987,-0.067005,-0.020914,-0.026879,0.004732,-0.026272,-0.007918,0.041024,0.030091,0.094012,-0.011227,0.074026,0.103852,-0.057710,-0.023862,-0.049002,-0.036862,0.054488,0.041895,0.022709,-0.054407,0.046822,0.079697,0.011529,0.012822,0.023991,0.005750,0.022059,-0.010626,-0.001139,-0.038968,-0.047409,0.005231,0.039488,-0.026971,0.038308,0.005497,0.043843,0.033286,0.066283,-0.090781,0.001885,-0.040152,0.017480,0.015877,-0.032509,-0.035167,-0.003784,-0.123003,-0.016040,0.036961,0.005584,-0.084428,0.062699,-0.031926,-0.069984,2.658555e-33,0.070547,0.010892,-0.020939,0.088458,0.025692,-0.

Many embeddings appear thousands of times, showing that different texts are mapped to the same vector often due to identical or semantically similar inputs. Skipping deduplication can result in dense or isolated clusters, while removing duplicates reduces noise, improves clustering, and speeds up computation.


In [8]:
import numpy as np
# deduplicate by row
unique_embeddings, indices = np.unique(embeddings, axis=0, return_index=True)
unique_preprocessed_texts = [preprocessed_texts[i] for i in indices]

In [9]:
print(len(unique_preprocessed_texts))

638497


# CPU BERTopic

In [10]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "1"

## Load the scikit-learn Libraries

In [11]:
from hdbscan import HDBSCAN
from umap import UMAP
from bertopic import BERTopic

## Create UMAP and HDBSCAN Instances

In [12]:
# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0, random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size = 50, min_samples=10, gen_min_span_tree=True, prediction_data=True)

## Build a BERTopic Model

In [13]:
%%time
# Pass the above models to be used in BERTopic
topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model)
topics, probs = topic_model.fit_transform(unique_preprocessed_texts)

CPU times: user 1h 32min 48s, sys: 1min 32s, total: 1h 34min 20s
Wall time: 11min


# GPU BERTopic

## Load the cuML Libraries

In [14]:
%load_ext cuml.accel

from hdbscan import HDBSCAN
from umap import UMAP
from bertopic import BERTopic

cuML: Accelerator installed.


## Create UMAP and HDBSCAN Instances

In [15]:
# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0, random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=50, min_samples=10, gen_min_span_tree=True, prediction_data=True)

build_algo set to brute_force_knn because random_state is given


## Build a BERTopic Model

In [16]:
%%time
# Pass the above models to be used in BERTopic
topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model)
topics, probs = topic_model.fit_transform(unique_preprocessed_texts)

CPU times: user 14min 19s, sys: 1min 30s, total: 15min 49s
Wall time: 6min 27s


## Get Topic ID and Topic Keywords

In [17]:
# Get the topic keywords
topic_info = topic_model.get_topic_info()  # Returns a DataFrame with topic details
topic_info  # Includes Topic ID and top keywords

,Topic,Count,Name,Representation,Representative_Docs
0,-1,419595,-1_hair_it_and_my,"[hair, it, and, my, the, this, to, is, for, in]",[like another reviewer this was not an item i ...
1,0,16057,0_nails_nail_polish_gel,"[nails, nail, polish, gel, coat, polishes, coa...",[yes many and most items have some type of che...
2,1,8725,1_lashes_mascara_eyelashes_lash,"[lashes, mascara, eyelashes, lash, magnetic, e...","[i love these lashes they are perfect, this is..."
3,2,7724,2_brushes_brush_bristles_handle,"[brushes, brush, bristles, handle, soft, brist...","[nice brushes, great hair brush, love this bru..."
4,3,7528,3_el_de_muy_es,"[el, de, muy, es, que, la, lo, para, se, en]",[ese producto es muy bueno y es muy usado en b...
...,...,...,...,...,...
459,458,51,458_thread_threads_spools_embroidery,"[thread, threads, spools, embroidery, gxmm, ne...",[it didnt help regular thread was a lot better...
460,459,51,459_gloss_nectar_clear_color,"[gloss, nectar, clear, color, sticky, finish, ...","[perfect gloss, great gloss, love this gloss]"
461,460,51,460_thanks_thank_aaaaaaaaaaall_thanku,"[thanks, thank, aaaaaaaaaaall, thanku, wonderf...","[thanks, thanks, thanks]"
462,461,51,461_satin_polyester_silk_material,"[satin, polyester, silk, material, inside, say...","[my hair feels like satin every time, they all..."


## Visualize Document DataMap

In [ ]:
topic_model.visualize_document_datamap(
    unique_preprocessed_texts,
    interactive=True
)

<b>Note</b>: The Document DataMap visualization has been removed due to its large size. You can still generate it by running the notebook locally.